---
layout: post

title: 백준 문제 14499 with python, 구현, 주사위 굴리기

category: Algorithms
---

# 백준 문제 중 14499번

### 문제

크기가 N×M인 지도가 존재한다. 지도의 오른쪽은 동쪽, 위쪽은 북쪽이다. 이 지도의 위에 주사위가 하나 놓여져 있으며, 주사위의 전개도는 아래와 같다. 지도의 좌표는 (r, c)로 나타내며, r는 북쪽으로부터 떨어진 칸의 개수, c는 서쪽으로부터 떨어진 칸의 개수이다.

      2
    4 1 3
      5
      6

주사위는 지도 위에 윗 면이 1이고, 동쪽을 바라보는 방향이 3인 상태로 놓여져 있으며, 놓여져 있는 곳의 좌표는 (x, y) 이다. 가장 처음에 주사위에는 모든 면에 0이 적혀져 있다.

지도의 각 칸에는 정수가 하나씩 쓰여져 있다. 주사위를 굴렸을 때, 이동한 칸에 쓰여 있는 수가 0이면, 주사위의 바닥면에 쓰여 있는 수가 칸에 복사된다. 0이 아닌 경우에는 칸에 쓰여 있는 수가 주사위의 바닥면으로 복사되며, 칸에 쓰여 있는 수는 0이 된다.

주사위를 놓은 곳의 좌표와 이동시키는 명령이 주어졌을 때, 주사위가 이동했을 때 마다 상단에 쓰여 있는 값을 구하는 프로그램을 작성하시오.

주사위는 지도의 바깥으로 이동시킬 수 없다. 만약 바깥으로 이동시키려고 하는 경우에는 해당 명령을 무시해야 하며, 출력도 하면 안 된다.

### 입력

첫째 줄에 지도의 세로 크기 N, 가로 크기 M (1 ≤ N, M ≤ 20), 주사위를 놓은 곳의 좌표 x, y(0 ≤ x ≤ N-1, 0 ≤ y ≤ M-1), 그리고 명령의 개수 K (1 ≤ K ≤ 1,000)가 주어진다.

둘째 줄부터 N개의 줄에 지도에 쓰여 있는 수가 북쪽부터 남쪽으로, 각 줄은 서쪽부터 동쪽 순서대로 주어진다. 주사위를 놓은 칸에 쓰여 있는 수는 항상 0이다. 지도의 각 칸에 쓰여 있는 수는 10 미만의 자연수 또는 0이다.

마지막 줄에는 이동하는 명령이 순서대로 주어진다. 동쪽은 1, 서쪽은 2, 북쪽은 3, 남쪽은 4로 주어진다.

### 출력

이동할 때마다 주사위의 윗 면에 쓰여 있는 수를 출력한다. 만약 바깥으로 이동시키려고 하는 경우에는 해당 명령을 무시해야 하며, 출력도 하면 안 된다.

---

문제를 접하고 처음에 재귀적으로 dfs로 접근 했는데 Recursion error가 발생했다. 그래서 이를 반복구조로 개선해 해결했다.

이 문제의 핵심은 주사위를 굴릴때마다 주사위의 윗면, 아랫면, 오른쪽면 모두가 변한다는 것이다. 이동 할 때 마다 이를 어떻게 고려할지가 관건이다.

우선 코드를 짜기 앞서 알아야 할 것은 정상적인 주사위의 대표적인 규칙인 마주보는 면의 합이 7이라는 것이다. 그러므로 6개의 면 중 3개면의 상태만 알 수 있다면 나머지 3개의 면은 자연스럽게 알 수 있다. 이 규칙을 이용해 윗면, 오른쪽면, 앞면을 각각 top, right, front라는 변수에 저장하고 이를 주사위의 상태를 나타내기 위해 활용할 것이다.

코드의 흐름은 이렇다.
1. 각 주사위 면과 면에 저장될 값을 나타낼 딕셔너리 dice를 선언하고 모든 값을 0으로 초기화 한다.  
2. 각 방향을 나타내는 딕셔너리 dir를 선언한다.
3. 주사위의 위치 x,y 주사의의 3면의 상태 top, right, front를 입력받아 명령을 수행할 함수 roll을 선언한다.
4. roll은 내부적으로 주어진 명령을 순차적으로 실향하는데, 주어진 명령으로 이동해야할 좌표가 인덱스를 벗어나지 않는다면 이동을하고, 주사위의 top, right, front의 상태를 최신화 한다.
5. 이동된 주사위와 보드의 값을 주어진 조건에 맞게 조작한다.
6. 주사위 top면에 해당하는 값 dice[top]을 출력한다.
7. 만약 4번의 분기에서 인덱스를 벗어난다면 주사위를 굴리기 전 위치로 다시 이동한다.

In [68]:
n, m, x, y, k = map(int, input().split())
board = []
for _ in range(n):
    board.append(list(map(int, input().split())))
cmds = list(map(int, input().split()))

 # 주사의의 각면에 값을 저장할 딕셔너리 선언
dice = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0}

 # 각 방향을 나타내는 딕셔너리
dir = {1: [0, 1], 2: [0, -1], 3: [-1, 0], 4: [1, 0]}

def roll(x, y, top = 1, right = 3, front = 5):

    idx = 0
    
    while idx<k:    
        c = cmds[idx]
        prev_x, prev_y = x, y
        x, y = x + dir[c][0], y + dir[c][1]
        if 0 <= x < n and 0 <= y < m:
             # 방향에 따라 주사위 상태를 최신화      
            if c == 1:
                top, right, front = 7 - right, top, front
            elif c == 2:
                top, right, front = right, 7 - top, front
            elif c == 3:
                top, right, front = front, right, 7 - top
            elif c == 4:
                top, right, front = 7 - front, right, top

            if board[x][y] == 0:
                board[x][y] = dice[7 - top]
            else:
                dice[7 - top] = board[x][y]
                board[x][y] = 0 

            print(dice[top]) 
    
         # 인덱스를 벗어난다면 이전 위치로 돌아감      
        else:
            x, y = prev_x, prev_y       
        
        idx += 1
        


roll(x, y)

4 2 0 0 8
0 2
3 4
5 6
7 8
4 4 4 1 3 3 3 2
0
0
3
0
0
8
6
3
